Import dependencies

In [6]:
# import gym
import gym
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete

#import helpers
import numpy as np
import random
import os

#import stable baselines
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

Types of Spaces

In [131]:
Discrete(3).sample()

1

In [132]:
Box(0, 1, shape=(3,)).sample()

array([0.33654493, 0.7136001 , 0.6445912 ], dtype=float32)

In [133]:
Tuple((Discrete(3), Box(0,1, shape=(3,)))).sample()

(2, array([0.72962904, 0.27369156, 0.6415968 ], dtype=float32))

In [134]:
Dict({'height':Discrete(2), "speed":Box(0, 100, shape=(1,))}).sample()

OrderedDict([('height', 1), ('speed', array([17.37748], dtype=float32))])

In [135]:
MultiBinary(4).sample()

array([1, 0, 1, 0], dtype=int8)

In [136]:
MultiDiscrete([5,10,200]).sample()

array([ 1,  1, 24])

Building environment

In [7]:
# keep water temperature between 37 and 39 *C
class ShowerEnv(Env):
    def __init__(self):
        self.action_space = Discrete(3)
        self.observation_space = Box(low=np.array([0]), high=np.array([100])) # Box(low=0, high=100, shape=(1,))
        self.state = 38 + random.randint(-3, 3) # initial state
        self.shower_length = 60 # frames 

    def step(self, action):
        # apply temp adj
        self.state += action - 1    # increaser, do nothing or decrease temperature

        # decrease shower time
        self.shower_length -= 1

        # calculate reward
        if self.state >= 37 and self.state <= 39:
            reward = 1
        else:
            reward = -1
        
        if self.shower_length <= 0:
            done = True
        else:
            done = False


        self.state += random.randint(-1, 1)
        info = {}

        return self.state, reward, done, info
    
    def render(self):
        pass

    def reset(self):
        self.state = 38 + random.randint(-3, 3)
        self.shower_length = 60
        return self.state

In [8]:
env = ShowerEnv()

/home/kuba/.local/lib/python3.10/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


In [9]:
states = env.observation_space.shape[0]

In [10]:
actions = env.action_space.n

Test environment

In [141]:
episodes = 5
for episode in range(1, episodes + 1):
    observation = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action = env.action_space.sample()     # Using model here
        observation, reward, done, info = env.step(action)
        score += reward
    print("Episode {} Score {}".format(episode, score))
env.close()

Episode 1 Score -54
Episode 2 Score -60
Episode 3 Score -20
Episode 4 Score 2
Episode 5 Score -46


Create Deep Learning model with keras

In [3]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers.legacy import Adam

In [12]:
def build_model(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape = (1, states)))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [13]:
model = build_model(states, actions)

In [14]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 1)                 0         
                                                                 
 dense (Dense)               (None, 24)                48        
                                                                 
 dense_1 (Dense)             (None, 24)                600       
                                                                 
 dense_2 (Dense)             (None, 3)                 75        
                                                                 
Total params: 723
Trainable params: 723
Non-trainable params: 0
_________________________________________________________________


In [15]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy  # policy based rf
from rl.memory import SequentialMemory

In [17]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=5000, window_length=1)
    dqn = DQNAgent(model=model, policy=policy, memory=memory, nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [18]:
dqn = build_agent(model, actions)
dqn.compile(Adam(learning_rate=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=10000, visualize=False, verbose=1)

AttributeError: 'Sequential' object has no attribute '_compile_time_distribution_strategy'